
Warning: This colab may not work if tensorflow version 1 support from colab is removed by google

Source for port: https://github.com/guodongsanjianke/Neural-Network-for-solving-PDE


# Deep Galerkin Method (DGM)
### S1 = sigma(w1*x + b1)  Z(l) = sigma(u*x + w*S + b) l=1,...,L  G(l) = sigma(u*x + w*S + b) l=1,...,L  
### R(l) = sigma(u*x + w*S + b) l=1,...,L   H(l) = sigma(u*x + w*(S Hadamard R) + b)  l=1,...,L  
### S(L+1) = (1-G) Hadamard H + Z Hadamard S  f = w*S(L+1) + b  

In [1]:
%tensorflow_version 1.x

After that, `%tensorflow_version 1.x` will throw an error.

Your notebook should be updated to use Tensorflow 2.
See the guide at https://www.tensorflow.org/guide/migrate#migrate-from-tensorflow-1x-to-tensorflow-2.

TensorFlow 1.x selected.


In [2]:
#import needed packages
import tensorflow as tf
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

**input_dim:** dimensionality of imput data  
**output_dim:** number of output for LSTM layer  
**trans1, trans2 (str):** activation functions used inside the layer;  
one of: "tanh"(default), "relu" or "sigmoid"  
**u vectors:** weighting vectors for inputs original inputs x  
**w vectors:** weighting vectors for output of previous layer  
**S:** output of previous layer  
**X:** data input

In [3]:
# LSTM-like layer used in DGM
class LSTMLayer(tf.keras.layers.Layer):
    
    # constructor/initializer function (automatically called when new instance of class is created)
    def __init__(self, output_dim, input_dim, trans1 = "tanh", trans2 = "tanh"):
        
        super(LSTMLayer, self).__init__()
        
        self.output_dim = output_dim
        self.input_dim = input_dim
        
        if trans1 == "tanh":
            self.trans1 = tf.nn.tanh
        elif trans1 == "relu":
            self.trans1 = tf.nn.relu
        elif trans1 == "sigmoid":
            self.trans1 = tf.nn.sigmoid
        
        if trans2 == "tanh":
            self.trans2 = tf.nn.tanh
        elif trans2 == "relu":
            self.trans2 = tf.nn.relu
        elif trans2 == "sigmoid":
            self.trans2 = tf.nn.relu
        
        # u vectors (weighting vectors for inputs original inputs x)
        self.Uz = self.add_variable("Uz", shape=[self.input_dim, self.output_dim],
                                    initializer = tf.contrib.layers.xavier_initializer())
        self.Ug = self.add_variable("Ug", shape=[self.input_dim ,self.output_dim],
                                    initializer = tf.contrib.layers.xavier_initializer())
        self.Ur = self.add_variable("Ur", shape=[self.input_dim, self.output_dim],
                                    initializer = tf.contrib.layers.xavier_initializer())
        self.Uh = self.add_variable("Uh", shape=[self.input_dim, self.output_dim],
                                    initializer = tf.contrib.layers.xavier_initializer())
        
        # w vectors (weighting vectors for output of previous layer)        
        self.Wz = self.add_variable("Wz", shape=[self.output_dim, self.output_dim],
                                    initializer = tf.contrib.layers.xavier_initializer())
        self.Wg = self.add_variable("Wg", shape=[self.output_dim, self.output_dim],
                                    initializer = tf.contrib.layers.xavier_initializer())
        self.Wr = self.add_variable("Wr", shape=[self.output_dim, self.output_dim],
                                    initializer = tf.contrib.layers.xavier_initializer())
        self.Wh = self.add_variable("Wh", shape=[self.output_dim, self.output_dim],
                                    initializer = tf.contrib.layers.xavier_initializer())
        
        # bias vectors
        self.bz = self.add_variable("bz", shape=[1, self.output_dim])
        self.bg = self.add_variable("bg", shape=[1, self.output_dim])
        self.br = self.add_variable("br", shape=[1, self.output_dim])
        self.bh = self.add_variable("bh", shape=[1, self.output_dim])
    
    
    def call(self, S, X):

        Z = self.trans1(tf.add(tf.add(tf.matmul(X,self.Uz), tf.matmul(S,self.Wz)), self.bz))
        G = self.trans1(tf.add(tf.add(tf.matmul(X,self.Ug), tf.matmul(S, self.Wg)), self.bg))
        R = self.trans1(tf.add(tf.add(tf.matmul(X,self.Ur), tf.matmul(S, self.Wr)), self.br))
        
        H = self.trans2(tf.add(tf.add(tf.matmul(X,self.Uh), tf.matmul(tf.multiply(S, R), self.Wh)), self.bh))
        
        S_new = tf.add(tf.multiply(tf.subtract(tf.ones_like(G), G), H), tf.multiply(Z,S))
        
        return S_new

**input_dim:** dimensionality of input data  
**output_dim:** number of outputs for dense layer  
**transformation:** activation function used inside the layer; using None is equivalent to the identity map  
**w vectors:** weighting vectors for output of previous layer  
**X:** input to layer  

In [4]:
# Fully connected layer(dense)
class DenseLayer(tf.keras.layers.Layer):
    
    # constructor/initializer function (automatically called when new instance of class is created)
    def __init__(self, output_dim, input_dim, transformation=None):
        
        super(DenseLayer,self).__init__()
        self.output_dim = output_dim
        self.input_dim = input_dim

        self.W = self.add_variable("W", shape=[self.input_dim, self.output_dim],
                                   initializer = tf.contrib.layers.xavier_initializer())
        
        # bias vectors
        self.b = self.add_variable("b", shape=[1, self.output_dim])
        
        if transformation:
            if transformation == "tanh":
                self.transformation = tf.tanh
            elif transformation == "relu":
                self.transformation = tf.nn.relu
        else:
            self.transformation = transformation
    
    
    def call(self,X):
        
        S = tf.add(tf.matmul(X, self.W), self.b)
                
        if self.transformation:
            S = self.transformation(S)
        
        return S

**n_layers:** number of intermediate LSTM layers  
**input_dim:** spaital dimension of input data (excludes time dimension)  
**final_trans:** transformation used in final layer
define initial layer as fully connected
to account for time inputs we use input_dim+1 as the input dimension
**t:** sampled time inputs  
**x:** sampled space inputs  
Run the DGM model and obtain fitted function value at the inputs (t,x)

In [5]:
# Neural network architecture used in DGM

class DGMNet(tf.keras.Model):
    
    def __init__(self, layer_width, n_layers, input_dim, final_trans=None):
        
        super(DGMNet,self).__init__()
        
        self.initial_layer = DenseLayer(layer_width, input_dim+1, transformation = "tanh")
        
        self.n_layers = n_layers
        self.LSTMLayerList = []
                
        for _ in range(self.n_layers):
            self.LSTMLayerList.append(LSTMLayer(layer_width, input_dim+1))
        
        self.final_layer = DenseLayer(1, layer_width, transformation = final_trans)
    
    def call(self,t,x):

        X = tf.concat([t,x],1)
        
        # call initial layer
        S = self.initial_layer.call(X)
        
        # call intermediate LSTM layers
        for i in range(self.n_layers):
            S = self.LSTMLayerList[i].call(S,X)
        
        # call final LSTM layers
        result = self.final_layer.call(S)
        
        return result

Parameters

In [6]:
# OU process parameters（Ornstein-Uhlenbeck Process）
kappa = 0
theta = 0.5
sigma = 2

# mean and standard deviation for (normally distributed) process starting value
alpha = 0.0
beta = 1

# tenminal time
T = 1.0

# bounds of sampling region for space dimension, i.e. sampling will be done on
# [multipliter*Xlow, multiplier*Xhigh]
Xlow = -4.0
Xhigh = 4.0
x_multiplier = 2.0
t_multiplier = 1.5

# neural network parameters
num_layers = 3
nodes_per_layer = 50
learning_rate = 0.001

# Training parameters
sampling_stages = 500
steps_per_sample = 10

# Sampling parameters
nSim_t = 5
nSim_x_interior = 50
nSim_x_initial = 50

# Save options
saveName = 'FokkerPlanck'
saveFigure = False

**OU Simulation function(Ornstein-Uhlenbeck Process)**  
Simulate end point of Ornstein-Uhlenbeck process with normally distributed random starting value  
**alpha:** mean of random starting value  
**beta:** standard deviation of random starting value   
**theta:** mean reversion level    
**kappa:** mean reversion rate  
**sigma:** volatility  
**nSim:** number of simulations  
**T:** terminal time  

In [7]:
def simulateOU_GaussianStart(alpha, beta, theta, kappa, sigma, nSim, T):
    
    # simulate initial point based on normal distribution
    X0 = np.random.normal(loc = alpha, scale = beta, size = nSim)
    
    # mean and variance of OU endpoint
    m = theta + (X0 - theta) * np.exp(-kappa * T)
    v = np.sqrt(sigma**2 / (2 * kappa) * (1 - np.exp(-2*kappa*T)))
    
    # simulate endpoint
    Xt = np.random.normal(m,v)    
    
    return Xt

Sample time-space points from the function's domain;  
point are sampled uniformly on the interior of the domain, at the initial/terminal time points  
and along the spatial boundary at different time points.  
**nSim_t:** number of (interior) time points to sample  
**nSim_x_interior:** number of space points in the interior of the function's domain to sample  
**nSim_x_initial:** number of space points at initial time to sample (initial condition)

In [8]:
# Sampling function - random sample time-space pairs
def sampler(nSim_t, nSim_x_interior, nSim_x_initial):
    
    # Sampler1: domain interior
    t = np.random.uniform(low=0, high=T*t_multiplier, size=[nSim_t, 1])
    x_interior = np.random.uniform(low=Xlow*x_multiplier, high=Xhigh*x_multiplier, size=[nSim_x_interior, 1])
    
    # Sampler: spatial boundary
    # no spatial boundary condition for this problem 
    
    # Sampler3: initial/terminal condition
    x_initial = np.random.uniform(low=Xlow*1.5, high=Xhigh*1.5, size = [nSim_x_initial, 1])
    
    return t, x_interior, x_initial

Compute total loss for training.     
The loss is based o the PDE satisfied by the negative-exponential of the density and NOT the density   
itself, i.e. the u(t,x) in p(t,x) = exp(-u(t,x)) / c(t) where p is the density and c is the normalization constant.    
**model:** DGM model object   
**t:** sampled (interior) time points  
**x_interior:** sampled space points in the interior of the function's domain   
**x_initial:** sampled space points at initial time   
**nSim_t:** number of (interior) time points sampled (size of t)  
**alpha:** mean of normal distribution for process staring value  
**beta:** standard deviation of normal distribution for process starting value  


In [9]:
# Loss function for Fokker-Planck equation

def loss(model, t, x_interior, x_initial, nSim_t, alpha, beta):
    
    # Loss term1: PDE
    
    # initialize vector of losses
    losses_u = []
    
    # for each simulated interior time point
    for tIndex in range(nSim_t):
        
        curr_t = t[tIndex]
        t_vector = curr_t * tf.ones_like(x_interior)
        
        u    = model.call(t_vector, x_interior)
        u_t  = tf.gradients(u, t_vector)[0]
        u_x  = tf.gradients(u, x_interior)[0]
        u_xx = tf.gradients(u_x, x_interior)[0]

        psi_denominator = tf.reduce_sum(tf.exp(-u))
        psi = tf.reduce_sum( u_t*tf.exp(-u) ) / psi_denominator

        # PDE differential operator
        diff_f = -u_t + kappa - kappa*(x_interior- theta)*u_x - 0.5*sigma**2*(-u_xx + u_x**2) + psi
        
        # compute L2-norm of differential operator and attach to vector of losses
        currLoss = tf.reduce_mean(tf.square(diff_f)) 
        losses_u.append(currLoss)
    
    # average losses across sample time points 
    L1 = tf.add_n(losses_u) / nSim_t
    
    # Loss term2: boundary condition
    # no boundary condition for this problem
    
    # Loss term3: initial condition
    # compute negative-exponential of neural network-implied pdf at t = 0 i.e. the u in p = e^[-u(t,x)] / c(t)
    fitted_pdf = model.call(0*tf.ones_like(x_initial), x_initial)
    
    target_pdf  = 0.5*(x_initial - alpha)**2 / (beta**2)
    
    # average L2 error for initial distribution
    L3 = tf.reduce_mean(tf.square(fitted_pdf - target_pdf))

    return L1, L3

##### input(time, space domain interior, space domain at initial time)

In [10]:
# Set up network
model = DGMNet(nodes_per_layer, num_layers, 1)

t_tnsr = tf.placeholder(tf.float32, [None,1])
x_interior_tnsr = tf.placeholder(tf.float32, [None,1])
x_initial_tnsr = tf.placeholder(tf.float32, [None,1])

# loss 
L1_tnsr, L3_tnsr = loss(model, t_tnsr, x_interior_tnsr, x_initial_tnsr, nSim_t, alpha, beta)
loss_tnsr = L1_tnsr + L3_tnsr

u = model.call(t_tnsr, x_interior_tnsr)
p_unnorm = tf.exp(-u)

# set optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss_tnsr)

# initialize variables
init_op = tf.global_variables_initializer()

# open session
sess = tf.Session()
sess.run(init_op)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [11]:
#Train network
for i in range(sampling_stages):
    
    # sample uniformly from the required regions
    t, x_interior, x_initial = sampler(nSim_t, nSim_x_interior, nSim_x_initial)
    
    for j in range(steps_per_sample):
        loss,L1,L3,_ = sess.run([loss_tnsr, L1_tnsr, L3_tnsr, optimizer],
                                feed_dict = {t_tnsr:t, x_interior_tnsr:x_interior, x_initial_tnsr:x_initial})
        
    print(loss, L1, L3, i)

22.812868 5.436423 17.376446 0
10.718926 2.5900948 8.128832 1
4.222125 2.3220575 1.9000678 2
5.185042 1.5877342 3.5973077 3
2.230497 0.79645437 1.4340425 4
1.4961798 0.21687324 1.2793065 5
0.9004463 0.15370221 0.7467441 6
2.8749466 1.8999363 0.9750103 7
5.5001974 3.116194 2.3840034 8
3.5943904 1.9488782 1.6455123 9
1.2869014 0.36830148 0.91859984 10
1.7028573 0.7882127 0.9146446 11
0.6928726 0.07769261 0.61517996 12
1.0116818 0.40945527 0.6022265 13
0.36748284 0.064212315 0.30327052 14
1.318481 0.66494864 0.65353227 15
1.3227687 0.4206755 0.9020932 16
1.9980757 1.3705178 0.62755793 17
0.925474 0.53773135 0.38774264 18
0.5125839 0.15491436 0.35766953 19
0.43031222 0.22688413 0.20342807 20
0.47286168 0.100831866 0.3720298 21
0.33298072 0.22121067 0.11177004 22
0.29282647 0.12803052 0.16479595 23
0.113614395 0.02909561 0.08451878 24
0.1086739 0.063285016 0.04538889 25
0.80649495 0.44755998 0.35893497 26
0.38746822 0.15356258 0.23390564 27
0.15997104 0.06923182 0.09073922 28
1.2749097 0.90

In [12]:
saver = tf.train.Saver()
saver.save(sess, './FokkerPlack/' + saveName)

'./FokkerPlack/FokkerPlanck'